# Phase 5 — Deep Learning Model Development

Train FFNN, CNN, LSTM and Autoencoder. Use Keras Tuner for hyperparameter search (example placeholders included).

In [1]:
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, Flatten, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from pathlib import Path

# Load balanced training set
X = pd.read_csv("../data/processed/ml_balance/train_balanced.csv").values
y = pd.read_csv("../data/processed/ml_balance/train_balanced_labels.csv").values
# Ensure y is a flat Series

# ensure Series
if isinstance(y, pd.DataFrame) and y.shape[1] == 1:
    y = y.iloc[:, 0]



# Keep test set aside for final evaluation
X_test = pd.read_csv("../data/processed/ml_balance/test.csv").values
y_test = pd.read_csv("../data/processed/ml_balance/test_labels.csv").values
# Ensure y is a flat Series
if isinstance(y, pd.DataFrame) and y.shape[1] == 1:
    y = y.iloc[:, 0]                                 # convert to Series

num_classes = len(np.unique(y))
y_cat = to_categorical(y, num_classes)

# Train/val split
X_train, X_val, y_train, y_val = train_test_split(X, y_cat, test_size=0.2, stratify=y, random_state=42)

MODEL_DIR = Path("../trained_models/dl_models")
MODEL_DIR.mkdir(parents=True, exist_ok=True)


print(f"Deep learning models will be saved to {MODEL_DIR}")


Deep learning models will be saved to ..\trained_models\dl_models


In [4]:
# ---------------- FFNN ----------------
print("---------------- FFNN ----------------")
ffnn = Sequential([
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])
ffnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_ffnn = ffnn.fit(X_train, y_train, 
                        epochs=30, 
                        batch_size=256, 
                        validation_data=(X_val, y_val))

ffnn.save(MODEL_DIR/"final_ffnn.keras")

---------------- FFNN ----------------
Epoch 1/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 26s 10ms/step - accuracy: 0.9259 - loss: 0.1895 - val_accuracy: 0.9464 - val_loss: 0.1208
Epoch 2/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 23s 10ms/step - accuracy: 0.9445 - loss: 0.1279 - val_accuracy: 0.9486 - val_loss: 0.1090
Epoch 3/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 23s 10ms/step - accuracy: 0.9472 - loss: 0.1191 - val_accuracy: 0.9551 - val_loss: 0.1044
Epoch 4/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 25s 11ms/step - accuracy: 0.9495 - loss: 0.1144 - val_accuracy: 0.9543 - val_loss: 0.1055
Epoch 5/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 38s 9ms/step - accuracy: 0.9518 - loss: 0.1093 - val_accuracy: 0.9552 - val_loss: 0.0980
Epoch 6/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 23s 10ms/step - accuracy: 0.9529 - loss: 0.1057 - val_accuracy: 0.9547 - val_loss: 0.0943
Epoch 7/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 40s 10ms/step - accuracy: 0.9537 - loss: 0.1040 - val_accuracy: 0.9531 - val_loss: 0.0967
Epoch 8/30
2293/2293 ━━━━━━━━━━━━━━━━━

In [ ]:
# ---------------- CNN ----------------
print("---------------- CNN ----------------")
X_train_cnn = np.expand_dims(X_train, -1)
X_val_cnn = np.expand_dims(X_val, -1)
X_test_cnn = np.expand_dims(X_test, -1)

cnn = Sequential([
    Conv1D(128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    Conv1D(64, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(num_classes, activation='softmax')
])

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_cnn = cnn.fit(X_train_cnn, y_train, 
                      epochs=30, 
                      batch_size=256, 
                      validation_data=(X_val_cnn, y_val))

cnn.save(MODEL_DIR/"final_cnn.keras")

---------------- CNN ----------------
Epoch 1/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 137s 58ms/step - accuracy: 0.9355 - loss: 0.1591 - val_accuracy: 0.9526 - val_loss: 0.1041
Epoch 2/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 134s 59ms/step - accuracy: 0.9493 - loss: 0.1118 - val_accuracy: 0.9545 - val_loss: 0.0972
Epoch 3/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 136s 59ms/step - accuracy: 0.9526 - loss: 0.1043 - val_accuracy: 0.9598 - val_loss: 0.0935
Epoch 4/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 175s 76ms/step - accuracy: 0.9556 - loss: 0.0976 - val_accuracy: 0.9592 - val_loss: 0.0879
Epoch 5/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 142s 62ms/step - accuracy: 0.9575 - loss: 0.0945 - val_accuracy: 0.9634 - val_loss: 0.0835
Epoch 6/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 224s 97ms/step - accuracy: 0.9599 - loss: 0.0902 - val_accuracy: 0.8693 - val_loss: 0.3710
Epoch 7/30
2293/2293 ━━━━━━━━━━━━━━━━━━━━ 182s 79ms/step - accuracy: 0.9623 - loss: 0.0866 - val_accuracy: 0.9724 - val_loss: 0.0773
Epoch 8/30
2293/2293 ━━━━━━━━━━

In [ ]:
# ---------------- LSTM ----------------
print("---------------- LSTM ----------------")
X_train_lstm = np.expand_dims(X_train, -1)
X_val_lstm = np.expand_dims(X_val, -1)
X_test_lstm = np.expand_dims(X_test, -1)

lstm = Sequential([
    LSTM(256, input_shape=(X_train_lstm.shape[1],1), return_sequences=False),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])
lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_lstm = lstm.fit(X_train_lstm, y_train, 
                        epochs=30, 
                        batch_size=256, 
                        validation_data=(X_val_lstm, y_val))

lstm.save(MODEL_DIR/"final_lstm.keras")


In [ ]:
# ---------------- Autoencoder ----------------
print("---------------- Autoencoder ----------------")
input_dim = X.shape[1]
autoencoder = Sequential([
    Dense(256, activation='relu', input_shape=(input_dim,)),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(input_dim, activation='sigmoid')
])

autoencoder.compile(optimizer='adam', loss='mse')
history_ae = autoencoder.fit(X_train, X_train, 
                             epochs=20, 
                             batch_size=256, 
                             validation_data=(X_val, X_val))

autoencoder.save(MODEL_DIR/"final_autoencoder.keras")